In [25]:
#numpy, pandas, streamlitのパッケージインストール
import numpy as np
import pandas as pd
!pip install streamlit

!pip install gensim
from google.colab import drive
import re
import csv


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
#テキスト入力
!pip install streamlit
!pip install jedi

import streamlit as st
import time
from PIL import Image

app3.pyへ

#そのまま

形態素解析

In [27]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aptitude is already the newest version (0.8.13-3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-3)
git is already installed at the requested version (1:2.34.1-1ubuntu1.10)
make is already installed at the requested version (4.3-4.1build1)
curl is already installed at the requested version (7.81.0-1ubuntu1.15)
xz-utils is already installed at the requested version (5.2.5-2ubuntu1)
file is already installed at the requested version (1:5.41-3ubuntu0.1)
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the re

In [28]:
#!pip install janome

上の f = f.readlines()[2:]でー2して文章が空になってた可能性有。

app3.pyへ

形態素解析

x_Chart N
C
ベ
ン
ダ

学習用データ準備

モデル作成

model保存
#モデル置き換え

#streamlitに印字されている？

文書比較

app3.pyへ

TypeError: can only join an iterable

#app3.pyへ

#関数化

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

chart

ローカルダウンロード

#この中に!pip install streamlitがなくともipynbに記載があればOK

In [29]:
%%writefile app4.py
# %% [markdown]
# # スケジュール算出プログラム

# %% [markdown]
# ## 1.ライブラリのインポート
# pandasは表の処理

# %%
import random
import datetime
import numpy as np
import pandas as pd
from array import array


# %% [markdown]
# ## 2.時間のローカライズ

# %%
def read_excels():

    excel_path = '/content/drive/MyDrive/data/f_iden_3.xlsm'



    off_day_df  = pd.read_excel(excel_path, sheet_name="yukyu")                         # 休みの希望表
    skill_df    = pd.read_excel(excel_path, sheet_name="skill", header=[0, 1])          # スキル表
    due_date_df = pd.read_excel(excel_path, sheet_name="calendar")                      # 納期表
    start_day:pd.Series = pd.read_excel(excel_path, sheet_name="start")

    # 空データ(NaN)の置換(0)
    off_day_df.fillna(0, inplace=True)

    due_date_df.fillna(0, inplace=True)

    # データの置換
    off_day_df.replace("◎", 2, inplace=True)                                            # 希望休（◎）を 2 に置換
    due_date_df.replace("☆", 3, inplace=True)                                          # 納期（☆）を 3 に置換

    # 列ごとのデータ型(dtype)を、intに変更しておく
    off_day_df[off_day_df.columns[2:]] = off_day_df[off_day_df.columns[2:]].astype("int64")
    due_date_df[due_date_df.columns[5:]] = due_date_df[due_date_df.columns[5:]].astype("int64")   #スキルグループ→工程による列追加

    # 品番の抽出
    product_num = due_date_df.loc[:, "品番"]

    # 土日の削除
    # データの中から土日の日付を取り出す
    off_days = [date for date in due_date_df.columns[5:] if date.strftime("%a") in ["Sat","Sun"]]

    # 土日の日付を削除する
    off_day_df.drop(columns=off_days, inplace=True)
    due_date_df.drop(columns=off_days, inplace=True)

    return off_day_df, skill_df, due_date_df, product_num , start_day

# %%
off_day_df, skill_df, due_date_df, product_num , start_day = read_excels()

# %% [markdown]
# ## 3.第1世代関数

# %%
def first_gen(off_day_df: pd.DataFrame, skill_df: pd.DataFrame, due_date_df: pd.DataFrame, product_num: pd.Series):

    # 調整後の納期表（納期表のコピー）
    adjusted_due_date_df = due_date_df.copy()


    # 作業者の割り振り　と　作業者の希望休を調整納期表にコピー
    worker_list = []
    # indexごとにrowを取り出して、作業者をランダムに割り振る
    for index, row in adjusted_due_date_df.iterrows():



        workers: pd.DataFrame = skill_df.loc[:, (["スキルグループ"], [row["スキルグループ"]])]
        worker = workers.dropna().sample().values[0, 0]

        worker_list.append(worker)
        worker_off_days: pd.DataFrame = off_day_df.loc[off_day_df["設備"].isin([worker]), off_day_df.columns[2:]]                   # 作業者の希望休を抽出


        adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[5:]] = worker_off_days.values

    # 割り当てたワーカーを adjusted_due_date_df へ「作業者」列として追加（※ここから adjusted_due_date_df が一列増えるので注意）
    adjusted_due_date_df.insert(3, "作業者", worker_list)


    # 作業者の希望休に合わせて、納期を調整
    after_index = None                                                                                                              # 後工程の index を保持する

    for index, row in adjusted_due_date_df[::-1].iterrows():                                                                        # due_date_df を逆順で行ごとに処理

        # 現工程の納期を抽出
        process: pd.Series = due_date_df.loc[index, due_date_df.columns[5:]]

        process_due_date = process[process.isin([3])]                                                                               # 現工程の納期
        # print("入力納期:", process_due_date.index[0])
        # print("入力納期process_due_date:", process_due_date)

        if after_index != None:

            after_process_df: pd.DataFrame = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == row["製造番号"]]                               # 同じ製造番号のタスクを取り出す。


            print("[INFO]: 工程:", row["工程"])


            if row["工程"] + 1 in after_process_df["工程"].values:

                print("後工程あり")


                after_index = after_process_df[after_process_df["工程"].isin([row["工程"] + 1])].index[0]
                #print("after_index :", after_index)

                # 後工程の納期を抽出
                after_process: pd.Series = adjusted_due_date_df.loc[after_index, adjusted_due_date_df.columns[6:]]                 #スキルグループ→工程による列追加
                after_process_due_date = after_process[after_process.isin([3])]                                                     # 後工程の納期
                #print("後工程の納期:", after_process_due_date.index[0])


                if not process_due_date.index[0] < after_process_due_date.index[0]:

                    # 納期を１営業日早めて調整する（この時点では、有休をチェックしてないので決定しない）
                    before_days:pd.DataFrame = due_date_df.loc[[index], due_date_df.columns[5]:after_process_due_date.index[0]]     # 現在の納期より前のデータを抽出

                    new_due_date = before_days.columns[-2]                                                                          # 納期を１営業日早める
                    #print("後工程を考慮した納期調整:", new_due_date)

                    due_date_df.loc[index, process_due_date.index[0]] = 0                                                           # 現在の納期を納期表から削除
                    due_date_df.loc[index, new_due_date] = process_due_date.values[0]                                               # 新しい納期を納期表に追加

                    process: pd.Seies = due_date_df.loc[index, due_date_df.columns[5:]]

                    process_due_date = process[process.isin([3])]                                                                   # 現工程の納期
                    print("現工程の納期:", process_due_date.index[0])

            else:

                print("後工程なし:")

        else:

            print("後工程なし（1つ目のデータ）:")


        # 希望休に合わせた調整
        # 現在の納期より前のデータを抽出
        before_days: pd.DataFrame = adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[6]:process_due_date.index[0]] #スキルグループ→工程による列追加


        for new_due_date in before_days.columns[::-1]:

            # 納期を１日早める(-1日)
            #print("納期候補:", new_due_date)

            # その日が希望休でない場合(0 のとき)　→　納期決定　→　ループ終了
            if adjusted_due_date_df.loc[index, new_due_date] == 0:

                # 納期が確定するのはココだけ。valuesで[3]が入るindexと配列指定。values[0]で3指定。
                #print("納期決定:", new_due_date)
                adjusted_due_date_df.loc[index, new_due_date] = process_due_date.values[0]
                break

        after_index = index
        #print("-" * 100)              # デバッグ用（ループごとに横線を表示）

    return adjusted_due_date_df


# %%
#受取
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

# %%
#第1世代

# %% [markdown]
# ## ※　データのルール
# 製造番号は必ず連番

# %%
adjusted_due_date_df[adjusted_due_date_df["製造番号"] == 1].iloc[:, :15]

# %% [markdown]
# ## 4.納期をランダムで振り直し（再帰処理）
# ※　この処理中は希望休を考慮しない
#

# %% [markdown]
# ### 開始日

# %% [markdown]
#

# %%
start_date = datetime.datetime(2023, 6, 2)

# %% [markdown]
# ### 納期のランダム振り分け関数

# %%
def generate_random_dates(
        start_date: datetime.datetime,
        end_date: datetime.datetime):

    start_date = due_date_df.loc[:, start_date:].columns[1]

    working_days = due_date_df.loc[:, start_date:end_date].columns
    #print(working_days)

    random_num = random.randint(0, working_days.shape[0]-1)
    random_date = working_days[random_num]
    #print(random_date)
    #print("-" * 50, "納期ランダム割り振り完了","-" * 50)

    return random_date

# %% [markdown]
# ### 同じ製造番号のタスクをランダムにリスケする関数（これが再帰関数：関数内の処理に自分自身の呼び出しがある）

# %%
def random_due_date(
        start_date: datetime.datetime,
        process_df: pd.DataFrame,
        process_num: int):

    #print("process_num :", process_num, "-" * 100)
    #print('process_df',process_df.loc[[process_df.index[process_num]], process_df.columns[:5]])

    process: pd.Series = process_df.loc[process_df.index[process_num], process_df.columns[5:]]              # 現工程の納期表データ
    deadline = process[process.isin([3])]                                                                   # 現工程の納期
    deadline_date: datetime.datetime = deadline.index[0]                                                    # その工程のデッドライン
    #print("process_deadline :", deadline_date)


    if process_num != 0:

        start_date = random_due_date(start_date, process_df.loc[process_df.index[:-1], :], process_num-1)   #（再帰処理）

    new_due_date = generate_random_dates(start_date, deadline_date)                                         # 開始日～デッドライン　の期間からランダムに日付を選択

    due_date_df.loc[process_df.index[process_num], deadline.index[0]] = 0                                   # 現在の納期を納期表から削除
    due_date_df.loc[process_df.index[process_num], new_due_date] = deadline.values[0]

    return new_due_date



# %% [markdown]
# ### タスクを製造番号ごとに処理する関数

# %%
def change_due_date():

    serial_number = due_date_df.loc[:, "製造番号"].max()
    #print('serial_number',serial_number)

    # 製造番号ごとに繰り返す
    for i in range(1, serial_number + 1):

        # 同じ製造番号のタスクを取り出す。
        process_df = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == i]

        #print('process_df.shape',process_df.shape)
        #print('process_df.loc[:, process_df.columns].isin([3])')



        random_due_date(start_date, process_df, process_df.shape[0]-1)                                          # 製造番号ごとに納期のランダム割り当て（再帰関数）

        #print('タスク',process_df)
        #print('process_df.shape[0]',process_df.shape[0])
        #print("*" * 200)

### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。



# %%
# 呼び出し
change_due_date()

# %% [markdown]
# ### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。

# %%
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

# %% [markdown]
# #### これでランダムな納期かつ希望休の調整ができる

# %% [markdown]
# ##### 希望休表

# %% [markdown]
# ##### 希望休調整前(ランダム)

# %% [markdown]
# ##### 希望休調整後

# %%
adjusted_due_date_df_copy= adjusted_due_date_df.copy()
adjusted_due_date_df_copy=adjusted_due_date_df_copy.iloc[:,5:]
adjusted_due_date_df_copy.reset_index(inplace=True,drop=True)
#行と列の数を取り出す
sh3=adjusted_due_date_df_copy.shape
adjusted_due_date_df_copy.columns=range(sh3[1])


# %%
#45変換
def pre_evalution(adjusted_due_date_df):
    adjusted_due_date_df2=adjusted_due_date_df.iloc[:,6:] #スキルグループ→工程による列追加

    return adjusted_due_date_df2

# %%


# %%
#受取
adjusted_due_date_df2= pre_evalution(adjusted_due_date_df)

# %%
x=adjusted_due_date_df2.replace(3,'☆').replace(2,'◎').replace(0,'')
#x.to_excel('AS_tochu.xlsx')
#x.to_excel('/content/drive/MyDrive/AS_tochu1.xlsx')
x.to_excel( '/content/drive/MyDrive/data/AS_tochu.xlsx')




Overwriting app4.py


In [30]:
#st内容はappへ記載。

In [31]:
from google.colab import files
#files.view("/content")
#files.view("app2.py")

# files.view("/content")
# files.view("app3.py")
files.view("/content")
files.view("app4.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
#import streamlit as st
#st.write(hyouji_list)
#t.table(hyouji_list)

In [33]:
st.title('生産計画')
st.write('スケジュール')
# if st.button('表示する'):
#     df2 = pd.read_excel(r'/content/drive/MyDrive/data/AS_tochu.xlsx',sheet_name='Sheet1')
#     st.table(df2)
df2 = pd.read_excel(r'/content/drive/MyDrive/data/AS_tochu.xlsx',sheet_name='Sheet1')
st.table(df2)

/usr/local/lib/python3.10/dist-packages/streamlit/type_util.py:832: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = pa.Table.from_pandas(df)


DeltaGenerator()

In [24]:
!streamlit config show


 # Below are all the sections and options you can have in
 ~/.streamlit/config.toml.
 

[global]

# By default, Streamlit checks if the Python watchdog module is available
# and, if not, prints a warning asking for you to install it. The watchdog
# module is not required, but highly recommended. It improves Streamlit's
# ability to detect changes to files in your filesystem.

# If you'd like to turn off this warning, set this to True.

# Default: false
# DEPRECATED.
# global.disableWatchdogWarning has been deprecated and will be removed in a future version.
# This option will be removed on or after 2024-01-20.
# disableWatchdogWarning = false

# By default, Streamlit displays a warning when a user sets both a widget
# default value in the function defining the widget and a widget value via
# the widget's key in `st.session_state`.

# If you'd like to turn off this warning, set this to True.

# Default: false
# disableWidgetStateDuplicationWarning = false

# If True, will show a warnin

In [ ]:
!streamlit run app4.py & sleep 3 && npx localtunnel --port 8501



npx: installed 22 in 9.12s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.80.105:8501

your url is: https://good-guests-prove.loca.lt
後工程なし（1つ目のデータ）:
[INFO]: 工程: 3
後工程あり
現工程の納期: 2023-06-09 00:00:00
[INFO]: 工程: 2
後工程あり
現工程の納期: 2023-06-08 00:00:00
[INFO]: 工程: 1
後工程あり
現工程の納期: 2023-06-07 00:00:00
[INFO]: 工程: 3
後工程なし:
[INFO]: 工程: 2
後工程あり
現工程の納期: 2023-06-09 00:00:00
[INFO]: 工程: 1
後工程あり
現工程の納期: 2023-06-08 00:00:00
[INFO]: 工程: 3
後工程なし:
[INFO]: 工程: 2
後工程あり
現工程の納期: 2023-06-09 00:00:00
[INFO]: 工程: 1
後工程あり
現工程の納期: 2023-06-08 00:00:00
[INFO]: 工程: 2
後工程なし:
[INFO]: 工程: 1
後工程あり
現工程の納期: 2023-06-15 00:00:00
[INFO]: 工程: 3
後工程なし:
[INFO]: 工程: 2
後工程あり
現工程の納期: 2023-06-06 00:00:00
[INFO]: 工程: 1
後工程あり
現工程の納期: 2023-06-05 00:00:00
[INFO]: 工程: 1
後工程なし:
[INFO]: 工程: 3
後工程なし:
[INFO]: 工程: 2
後工程あり
現工程の納期: 2023-06-21 00:00:00
[INFO]: 工程: 1
後工程あり
現工程の納期: 2023-06-20 00:00:00
[INFO]: 工程: 1
後工程なし:
[INFO]: 工程: 3
後工程なし:
[INFO]: 工程: 2
後工程あり
現工程の納期